<a href="https://colab.research.google.com/github/pinoke98/Artificial_Intelligence/blob/main/Proyecto_Fox_and_Hounds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pydot
from IPython.display import Image, display
import queue
import numpy as np

In [ ]:
class Node ():
  def __init__(self, state,value,operators,operator=None, parent=None,objective=None):
    self.state= state
    self.value = value
    self.children = []
    self.parent=parent
    self.operator=operator
    self.objective=objective
    self.level=0
    self.operators=operators
    self.v=0

    
  def add_child(self, value, state, operator):
    node=type(self)(value=value, state=state, operator=operator,parent=self,operators=self.operators)
    node.level=node.parent.level+1
    self.children.append(node)
    return node
  
  def add_node_child(self, node):
    node.level=node.parent.level+1
    self.children.append(node)    
    return node

  #Devuelve todos los estados según los operadores aplicados
  def getchildrensZ(self):
    return [
        self.getStateZ(i) 
          if not self.repeatStatePath(self.getStateZ(i)) 
            else None for i, op in enumerate(self.operators)]
  
  def getchildrensH(self):
    return [
        self.getStateH(i) 
          if not self.repeatStatePath(self.getStateH(i)) 
            else None for i, op in enumerate(self.operators)]
    
  def getState(self, index):
    pass
  
  def __eq__(self, other):
    return self.state == other.state
 
  def __lt__(self, other):
    return self.f() < other.f()
   
  
  def repeatStatePath(self, state):
      n=self
      while n is not None and n.state!=state:
          n=n.parent
      return n is not None
    
  def pathObjective(self):
      n=self
      result=[]
      while n is not None:
          result.append(n)
          n=n.parent
      return result
  
  def heuristic(self):
    return 0
  
  def cost(self):
    return 1
  
  def f(self): 
    return self.cost()+self.heuristic()

  ### Crear método para criterio objetivo
  ### Por defecto vamos a poner que sea igual al estado objetivo, para cada caso se puede sobreescribir la función
  def isObjective(self):
    return 0
    #return (self.state==self.objetive.state)

In [ ]:
class Tree ():
  def __init__(self, root ,operators):
    self.root=root
    self.operators=operators

  def printPath(self,n):
    stack=n.pathObjective()
    path=stack.copy()
    while len(stack)!=0:
        node=stack.pop()
        if node.operator is not None:
            print(f'operador:  {self.operators[node.operator]} \t estado: {node.state}')
        else:
            print(f' {node.state}')
    return path

  def reinitRoot(self):
    self.root.operator=None
    self.root.parent=None
    self.root.objective=None
    self.root.children = []
    self.root.level=0

  def alphaBeta(self, depth):
    self.root.v= self.alphaBetaR(self.root, depth,float('-inf'),float('+inf'), True)
    if not self.root.isObjective():
      values =[c.v for c in self.root.children]
      maxvalue=max(values)
      index = values.index(maxvalue)
      return self.root.children[index]
    else:
      return self.root

  def alphaBetaR(self, node, depth,alpha,beta, maxPlayer):
    if depth==0 or node.isObjective():
      node.v = node.heuristic()
      #print(node.v)
      return node.heuristic()
    if maxPlayer:
      value=float('-inf')
      children = node.getchildrensZ()
      #print(children)
      for i,child in enumerate(children):
        if child is not None:
          newChild=type(self.root)(value=node.value+'-'+str(i),state=child,operator=i,parent=node, operators=node.operators,player=False)
          newChild=node.add_node_child(newChild)   
          value = max(value,self.alphaBetaR(newChild, depth-1, alpha,beta,False))
          alpha = max(alpha,value)
          if alpha>=beta:
            break
    else:
      value=float('inf')
      children = node.getchildrensH()
      #print(children)
      for i,child in enumerate(children):
        if child is not None:
          newChild=type(self.root)(value=node.value+'-'+str(i),state=child,operator=i,parent=node, operators=node.operators,player=True)
          newChild=node.add_node_child(newChild)   
          value = min(value,self.alphaBetaR(newChild, depth-1, alpha,beta,True))
          beta = min(beta,value)
          if alpha>=beta:
            break
    node.v=value
    return value

  ## Método para dibujar el árbol        
  def draw(self,path):
    graph = pydot.Dot(graph_type='graph')
    nodeGraph=pydot.Node(str(self.root.state)+"-"+str(0),
                          label=str(self.root.state),shape ="circle", 
                          style="filled", fillcolor="red")
    graph.add_node(nodeGraph)
    path.pop()
    return self.drawTreeRec(self.root,nodeGraph,graph,0,path.pop(),path)
  
  ## Método recursivo para dibujar el árbol  
  def drawTreeRec(self,r,rootGraph,graph,i,topPath,path):
    if r is not None:
      children=r.children
      for j,child in enumerate(children):
        i=i+1
        color="white"
        if topPath.value==child.value:
          if len(path)>0:topPath=path.pop()
          color='red'
        c=pydot.Node(child.value,label=str(child.state)+r"\n"+r"\n"+"f="+str(child.heuristic())+r"\n"+str(child.v), 
                      shape ="circle", style="filled", 
                      fillcolor=color)
        graph.add_node(c)
        graph.add_edge(pydot.Edge(rootGraph, c, 
                                  label=str(child.operator)+'('+str(child.cost())+')'))
        graph=self.drawTreeRec(child,c,graph,i,topPath,path)  # recursive call
      return graph
    else:
      return graph  

In [ ]:
class NodeFoxandHounds(Node ):
  ## Vamos a añadir el jugador, pues en dependencia del jugador se hace una cosa u otra.

  def __init__(self, player=True,**kwargs):
    super(NodeFoxandHounds, self).__init__(**kwargs)
    self.player=player
    self.ficha=1
    if player:
      self.v=float('-inf')
    else:
      self.v=float('inf')
  
  def heuristic(self):
    (x,y) = self.buscar(self.state,'Z')
    distanciaCentro = abs(4 - y)
    distanciaObjetivo = x+1
    return 1/(distanciaCentro+distanciaObjetivo)

  def buscar(self,state,dato):
    for i in range(len(initState)):
      for j in range(len(initState[0])):
        if state[i][j] == dato:
          return (i,j)

  def getStateZ(self,index): 
    state = self.state
    newState = [f.copy() for f in state]
    (x,y) = self.operators[index]
    (x1,y1) = self.buscar(self.state,'Z')
    if x1 == 7: 
      if y1 == 0:
        if (x == x1+1 and y == y1+1) and (state[x][y] == 0):
          newState[x1][y1] = 0
          newState[x][y] = 'Z'
      else:
        if (x == x1-1 and y == y1-1) and (state[x][y] == 0):
          newState[x1][y1] = 0
          newState[x][y] = 'Z'
        elif (x == x1+1 and y == y1+1) and (state[x][y] == 0):
          newState[x1][y1] = 0
          newState[x][y] = 'Z'
    elif y1 == 0 and x1 != 0:
      if (x == x1-1 and y == y1+1) and (state[x][y] == 0):
        newState[x1][y1] = 0
        newState[x][y] = 'Z'
      elif (x == x1+1 and y == y1+1) and (state[x][y] == 0):
        newState[x1][y1] = 0
        newState[x][y] = 'Z'
    elif y1 == 7 and x1 != 0:
      if (x == x1-1 and y == y1-1) and (state[x][y] == 0):
        newState[x1][y1] = 0
        newState[x][y] = 'Z'
      elif (x == x1+1 and y == y1-1) and (state[x][y] == 0):
        newState[x1][y1] = 0
        newState[x][y] = 'Z'
    elif y1 != 0 and x1 != 0 and y1 != 7 and x1 != 7:
      if (x == x1+1 and y == y1+1) and (state[x][y] == 0):
        newState[x1][y1] = 0
        newState[x][y] = 'Z'
      elif (x == x1+1 and y == y1-1) and (state[x][y] == 0):
        newState[x1][y1] = 0
        newState[x][y] = 'Z'
      elif (x == x1-1 and y == y1+1) and (state[x][y] == 0):
        newState[x1][y1] = 0
        newState[x][y] = 'Z'
      elif (x == x1-1 and y == y1-1) and (state[x][y] == 0):
        newState[x1][y1] = 0
        newState[x][y] = 'Z'
    return newState if state!=newState else None
    
  def getStateH(self,index):
    state = self.state
    newState = [f.copy() for f in state]
    (x,y) = self.operators[index]
    (x1,y1) = self.buscar(self.state,'Z')
    (x1,y1) = self.buscar(self.state,self.ficha)
    if x1 != 7: 
      if y1 == 7:
        if (x == x1+1 and y == y1-1) and (state[x][y] == 0):
          newState[x1][y1] = 0
          newState[x][y] = self.ficha
          self.ficha+1
      elif y1 == 0:
        if (x == x1+1 and y == y1+1) and (state[x][y] == 0):
          newState[x1][y1] = 0
          newState[x][y] = self.ficha
          self.ficha+1
      else:
        if (x == x1+1 and y == y1-1) and (state[x][y] == 0):
          newState[x1][y1] = 0
          newState[x][y] = self.ficha
          self.ficha+1
        elif (x == x1+1 and y == y1+1) and (state[x][y] == 0):
          newState[x1][y1] = 0
          newState[x][y] = self.ficha
          self.ficha+1
    return newState if state!=newState else None

  def cantMove(self,state):
    (x,y) = self.buscar(state,'Z')
    if x==len(state)-1 and y==0:
      if state[x-1][y+1] !=0:
        return True
    elif x==len(state)-1 and y==len(state[x])-1:
      if state[x-1][y-1] !=0:
        return True
    elif y==0:
      if state[x-1][y+1] !=0 and state[x+1][y+1] !=0:
        return True
    elif y==len(state[x])-1:
      if state[x-1][y-1] !=0 and state[x+1][y-1] !=0:
        return True
    elif x==len(state)-1:
      if state[x-1][y-1] !=0 and state[x-1][y+1] !=0:
        return True
    elif state[x-1][y-1] !=0 and state[x-1][y+1] !=0 and state[x+1][y-1] !=0 and state[x+1][y+1] !=0:
      return True
    return False
  
  def isObjective(self):
    (x,y) = self.buscar(self.state,'Z')
    if x == 0:
      return True;
    elif self.cantMove(self.state):
      return True
    return False

In [ ]:
class Juego:

  def printState(self,state):
    for i in state:
      aux = '|'
      for j in i:
        aux = aux +str(j)+'|'
      print(aux)
      #print('-----------------')
      

  def menu(self,initState):
    print("Elija la dificultad del juego: ")
    print("1.Facil")
    print("2.Moderado")
    print("3.Dificil")
    print("4.Terminar")
    x = input()
    dif = 0
    if int(x) == 1:
      dif = 2
    elif int(x) == 2:
      dif = 3
    elif int(x) == 3:
      dif = 4
    if dif == 0:
      return 0
    return self.turnos(dif,initState)
  
  def operadoresZ (self,estado): # devuelve todas las casillas negras
    fila = 0
    operadores = []
    while fila < len(estado):
      columna = 0
      while columna < len(estado[fila]):
        if fila %2 == 0 : 
          if (columna%2 != 0): 
            operadores.append((fila,columna))
        else: 
          if (columna%2 == 0): 
            operadores.append((fila,columna))
        columna = columna +1
      fila = fila +1
    return operadores
  
  def turnos(self,dif,initState):
    operators = self.operadoresZ(initState)
    state = NodeFoxandHounds(True,value = "Inicio",state = initState,operators=operators)
    self.printState(state.state)
    while not state.isObjective():
      #Introducir posicion
      temp = True
      aux = [1,2,3,4]
      while temp:
        print("Introducir que Hound se va a mover: 1-4")
        x = input()
        print("Introducir la direccion a la que desea moverse: ")
        print("1- Izquierda Inferior")
        print("2- Derecha inferior")
        y = input()
        try:
          x = int(x)
          y = int(y)
          #Cambiar el estado
          if x < 8 and y < 8:
            for i in range(len(state.state)):
              try:
                index = state.state[i].index(x)
                if index != None:
                  break
              except Exception:
                pass
            for j in aux:
              if x == j and y == 1:
                if index != 0:
                  if state.state[i+1][index-1] == 0:
                    state.state[i][index] = 0
                    state.state[i+1][index-1] = x
                    temp = False
                  else:
                    print("Ya casilla ya esta ocupada")
                else:
                  print("El movimiento no es posible")
              elif x == j and y == 2:
                if index != 7:
                  if state.state[i+1][index+1] == 0:
                    state.state[i][index] = 0
                    state.state[i+1][index+1] = x
                    temp = False
                  else:
                    print("Ya casilla ya esta ocupada")
                else:
                  print("El movimiento no es posible")
          else:
                print("La posicion ingresada no fue valida")
        except Exception:
          print("El dato ingresado no es valido")
      self.printState(state.state)
      #Juega IA(Ejecutar Alfabeta) y actualiza el estado
      #self.printState(state.state)
      treeMinMax= Tree(state,operators)
      #print(treeMinMax.root.state)
      state = treeMinMax.alphaBeta(dif)
      #Imprimir estado
      #print(state.heuristic())
      print("")
      self.printState(state.state)
      #print(state.value)
      state = NodeFoxandHounds(True,value = state.value,state = state.state,operators=operators)
  #return state.state

In [ ]:
def buscar(state,dato):
    for i in range(len(initState)):
      for j in range(len(initState[0])):
        if state[i][j] == dato:
          return (i,j)

In [ ]:
initState = [[ 0 , 1 , 0 , 2 , 0 , 3 , 0 , 4 ],
             [ 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],
             [ 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],
             [ 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],
             [ 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],
             [ 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],
             [ 0 , 'Z' , 0 , 0 , 0 , 0 , 0 , 0 ],
             [ 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ]]
juego = Juego()
juego.menu(initState)

Elija la dificultad del juego: 
1.Facil
2.Moderado
3.Dificil
4.Terminar
3
|0|1|0|2|0|3|0|4|
|0|0|0|0|0|0|0|0|
|0|0|0|0|0|0|0|0|
|0|0|0|0|0|0|0|0|
|0|0|0|0|0|0|0|0|
|0|0|0|0|0|0|0|0|
|0|Z|0|0|0|0|0|0|
|0|0|0|0|0|0|0|0|
Introducir que Hound se va a mover: 1-4
4
Introducir la direccion a la que desea moverse: 
1- Izquierda Inferior
2- Derecha inferior
1
|0|1|0|2|0|3|0|0|
|0|0|0|0|0|0|4|0|
|0|0|0|0|0|0|0|0|
|0|0|0|0|0|0|0|0|
|0|0|0|0|0|0|0|0|
|0|0|0|0|0|0|0|0|
|0|Z|0|0|0|0|0|0|
|0|0|0|0|0|0|0|0|

|0|1|0|2|0|3|0|0|
|0|0|0|0|0|0|4|0|
|0|0|0|0|0|0|0|0|
|0|0|0|0|0|0|0|0|
|0|0|0|0|0|0|0|0|
|0|0|Z|0|0|0|0|0|
|0|0|0|0|0|0|0|0|
|0|0|0|0|0|0|0|0|
Introducir que Hound se va a mover: 1-4
3
1
Introducir la direccion a la que desea moverse: 
1- Izquierda Inferior
2- Derecha inferior
|0|1|0|2|0|0|0|0|
|0|0|0|0|3|0|4|0|
|0|0|0|0|0|0|0|0|
|0|0|0|0|0|0|0|0|
|0|0|0|0|0|0|0|0|
|0|0|Z|0|0|0|0|0|
|0|0|0|0|0|0|0|0|
|0|0|0|0|0|0|0|0|

|0|1|0|2|0|0|0|0|
|0|0|0|0|3|0|4|0|
|0|0|0|0|0|0|0|0|
|0|0|0|0|0|0|0|0|
|0|0|0